Cette première cellule vous présente la manière avec laquelle j'ai obtenu les features/caractéristiques à partir des jeux de données de "textes"
Pour obtenir les textes (et les "filtrer") j'ai utiliser le script ./textDatasets/conversion.sh (qui prends, par exemple les fichiers FR_* les concatene, supprime les accents (en gardant les caracteres sans accents) et enfin transforme en minuscules).
J'ai utilisé les livres "open data" de la base du site Guttenberg Project (qui archive énormément de livres en texte, "txt" simples", tombés dans le domaine publique).

In [1]:
import numpy as np
import string
BlockSz=1000

# On procede de la même façon pour les differents languages
# Ouverture du fichier texte contenant les caracteres alphabetiques "brutes"
textGERMAN = open('./textDatasets/GERMAN')
# Lecture
DE=textGERMAN.read()
textGERMAN.close()
# Calcule du nombre de blocks (arrondi par defaut) de BlockSz caracteres
NbBlock=int(np.floor(len(DE)/BlockSz))
# Creation des tableaux de donnees textuelles et de features
DataDE=[]
FeaDE =[]
# Pour tous les indices de blocks ...
for index in range(NbBlock):
    # Exctraction des donnees texte du block
    DataDE.append(DE[index*BlockSz:(index+1)*BlockSz])
    # Comptage de chacun des caracteres
    countDE=[]
    for letter in string.ascii_lowercase:
        countDE.append(DataDE[index].count(letter))
    # Comptage de toutes les 26**2 paires de caracteres possibles
    for letter1 in string.ascii_lowercase:
        for letter2 in string.ascii_lowercase:
            countDE.append(DataDE[index].count(letter1+letter2))
    # Une fois le vecteur de décompte obtenu  .... on ajoute ce vecteur (contenant tous les decomptes) au tableaux des features
    FeaDE.append(countDE)

# On procede de la même façon pour les differents languages ....
textFRENCH = open('./textDatasets/FRENCH')
FR=textFRENCH.read()
textFRENCH.close()
NbBlock=int(np.floor(len(FR)/BlockSz))
DataFR=[]
FeaFR =[]
for index in range(NbBlock):
    DataFR.append(FR[index*BlockSz:(index+1)*BlockSz])
    countFR=[]
    for letter in string.ascii_lowercase:
        countFR.append(DataFR[index].count(letter))
    for letter1 in string.ascii_lowercase:
        for letter2 in string.ascii_lowercase:
            countFR.append(DataFR[index].count(letter1+letter2))
    FeaFR.append(countFR)

# On procede de la même façon pour les differents languages ....
textSPANISH = open('./textDatasets/SPANISH')
ES=textSPANISH.read()
textSPANISH.close()
NbBlock=int(np.floor(len(ES)/BlockSz))
DataES=[]
FeaES =[]
for index in range(NbBlock):
    DataES.append(ES[index*BlockSz:(index+1)*BlockSz])

    countES=[]
    for letter in string.ascii_lowercase:
        countES.append(DataES[index].count(letter))
    for letter1 in string.ascii_lowercase:
        for letter2 in string.ascii_lowercase:
            countES.append(DataES[index].count(letter1+letter2))
    FeaES.append(countES)

# On procede de la même façon pour les differents languages ....
textENGLISH = open('./textDatasets/ENGLISH')
EN=textENGLISH.read()
textENGLISH.close()
NbBlock=int(np.floor(len(EN)/BlockSz))
DataEN=[]
FeaEN =[]
for index in range(NbBlock):
    DataEN.append(EN[index*BlockSz:(index+1)*BlockSz])
    countEN=[]
    for letter in string.ascii_lowercase:
        countEN.append(DataEN[index].count(letter))
    for letter1 in string.ascii_lowercase:
        for letter2 in string.ascii_lowercase:
            countEN.append(DataEN[index].count(letter1+letter2))
    FeaEN.append(countEN)

In [2]:
# Ici on sauvegarde les caracteristiques, pour eviter de les recalculer à chaque fois ....
np.save('./FeaEN.npy', FeaEN)
np.save('./FeaFR', FeaFR)
np.save('./FeaDE', FeaDE)
np.save('./FeaES', FeaES)

In [3]:
# ... On peut simplement les loader (et commenter les cellules precedentes)
FeaEN = np.load('./FeaEN.npy')
FeaFR = np.load('./FeaFR.npy')
FeaDE = np.load('./FeaDE.npy')

FeaES = np.load('./FeaES.npy')

In [4]:
# Pour plus de "facilier de manipulation" on trasforme les donnes en "matrice numpy" et on affiche la dimension de la base de donnees.
FeaDE = np.array(FeaDE)
print(FeaDE.shape)

FeaFR = np.array(FeaFR)
print(FeaFR.shape)

FeaES = np.array(FeaES)
print(FeaES.shape)

FeaEN = np.array(FeaEN)
print(FeaEN.shape)

(4329, 702)
(4116, 702)
(4321, 702)
(4209, 702)


## Questions:
* 1) Réaliser une classification binaire en utilisant les méthodes linéaires suivantes: regression "ridge", LASSO et SVM (sans noyau !).
 * Pour ces méthodes vous devez faire une recherche du meilleur paramètre de régularisation ;
 * Vous devrez égalemement selectionner les deux languages de votre choix

On choisit de différencier les langages DE et FR.
On commence par définir nos matrices d'apprentissage et de test.

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
dfDE = pd.DataFrame(FeaDE)
dfDE["lang"]=0

dfFR = pd.DataFrame(FeaFR)
dfFR["lang"]=1

df = pd.concat([dfDE, dfFR])
df.head()
y = df.pop("lang")
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


On réalise une classification des données en utilisant une régression logistique (et non linéaire, comme indiqué, qui correspond à un problème de régression).

On commence par rechercher les meilleurs paramètres de régularisation, à l'aide de la librairie GridSearchCV de sklearn.


In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(solver="liblinear")
grid={
    "C": np.logspace(-1,1,10),
    "penalty": [
        "l1", # l1 : lasso
        "l2" # l2 : ridge
    ]
}

logreg_cv=GridSearchCV(logreg, grid, cv=10)
logreg_cv.fit(X_train,y_train)
logreg_cv.best_params_

{'C': 0.2782559402207124, 'penalty': 'l2'}

On en conclut que la régularisation qui résoud le mieux ce problème est une régularisation L2 (ridge), qui enlève les features les moins importantes.

Le coût optimal est de 0.27


In [7]:
from sklearn.metrics import accuracy_score

logreg = LogisticRegression(solver="liblinear", penalty=logreg_cv.best_params_["penalty"], C=logreg_cv.best_params_['C'])

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print(f"Accuracy on test : {accuracy_score(y_test, y_pred)}")

Accuracy on test : 0.9988158673771462


On réalise enfin une recherche des meilleurs paramètres à l'aide de GridSearchCV le SVM.


In [51]:
from sklearn import svm
parameters ={'C':np.logspace(-10, 10, 5)}

grid = {
    'C': np.logspace(-1,1,10),
    'penalty' : ["l1", "l2"]
}

grid_search = GridSearchCV(svm.LinearSVC(dual=False), grid, cv=10, verbose=1, n_jobs=-1)
grid_search.fit(X_test, y_test, )
grid_search.best_params_

Fitting 10 folds for each of 20 candidates, totalling 200 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    9.6s finished
C:\Users\Maitre\Anaconda3\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


{'C': 2.1544346900318834, 'penalty': 'l1'}

On obtient donc 2 meilleurs paramètres (penalty=l1 et C=2.15).

On entraine tout le modèle avec ces 2 paramètres.

In [57]:
SVM_model = svm.LinearSVC(dual=False, penalty=grid_search.best_params_["penalty"], C=grid_search.best_params_["C"], max_iter=10000)
SVM_model.fit(X_train, y_train)

y_pred = SVM_model.predict(X_test)
print(f"Accuracy on test : {accuracy_score(y_test, y_pred)}")

Accuracy on test : 0.9994079336885732


Le modèle de type SVM est globalement plus long à construire qu'un modèle de type régression logistique.
Cependant, les résultats obtenus sont légérements meilleurs.



## Questions:
* 2) Réaliser une classification binaire avec l'une des méthodes linéaire précédent en utilisant la réduction de dimension (ACP par exemple, ou une autre méthode)

On entraine et applique l'ACP sur le jeu de train. 


Par la suite, on applique l'ACP avec les dimensions précemment apprises sur le jeu de test, pour vérifier que les dimensions apprises sont bien généralisables.

In [58]:
from sklearn.decomposition import PCA
n_components=30
pca = PCA(n_components=n_components)
X_train_transform = pca.fit_transform(X_train)
X_test_transform = pca.transform(X_test)
X_train_transform.shape

(6756, 30)

In [59]:
logreg = LogisticRegression(solver="liblinear", penalty=logreg_cv.best_params_["penalty"], C=logreg_cv.best_params_['C'])

logreg.fit(X_train_transform, y_train)
y_pred = logreg.predict(X_test_transform)
print(f"Accuracy on test : {accuracy_score(y_test, y_pred)}")

Accuracy on test : 0.9988158673771462


Les résultats obtenus sont identiques à ceux obtenus précédemments. 

Cela veut dire que le jeu de données est de dimension trop importante et que ces dimensions peuvent être beaucoup réduites.

## Questions:
* 4. Réaliser une classification binaire en utilisant les SVM à noyau (comparer les performances obtenus avec un noyau Gaussien (rbf) et un noyau polynomial

In [75]:
grid = {
    "gamma" : ["auto", "scale"]
}

gridSearch = GridSearchCV(svm.SVC(kernel="rbf"), grid, cv=10, verbose=1, n_jobs=-1)
gridSearch.fit(X_train, y_train)
gridSearch.best_params_

{'gamma': 'scale'}

In [73]:
SVM_kernel = svm.SVC(kernel="rbf", gamma=gridSearch.best_params_["gamma"])
SVM_kernel.fit(X_train, y_train)
y_pred=SVM_kernel.predict(X_test)
accuracy_score(y_test, y_pred)

0.9964476021314387


On obtient des résultats qui sont légèrements moins bons que ceux obtenus précédemments avec un noyau linéaire.

On réalise la même opération avec un noyau polynomial


In [76]:
grid = {
    "gamma" : ["auto", "scale"]
}

gridSearch = GridSearchCV(svm.SVC(kernel="poly"), grid, cv=10, verbose=1, n_jobs=-1)
gridSearch.fit(X_train, y_train)
gridSearch.best_params_

Fitting 10 folds for each of 2 candidates, totalling 20 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    6.2s finished


{'gamma': 'auto'}

In [77]:
SVM_kernel = svm.SVC(kernel="rbf", gamma=gridSearch.best_params_["gamma"])
SVM_kernel.fit(X_train, y_train)
y_pred=SVM_kernel.predict(X_test)
accuracy_score(y_test, y_pred)

0.9674363528715216

Les résultats obtenus sont très en dessous des résultats précédents.

Ce type de noyau ne sera donc pas retenu pour la suite du travail.

## Questions:
* 5. Sur la base des résultats précédents, quelle est la méthode linéaire la plus adaptée à ce problème de classification


Sur la base des résultats précédents et en utilisant comme métrique de décision l'accuracy, la méthode la mieux adaptée pour résoudre ce problème de classification est le SVM linéaire.
Il s'agit de la seule méthode permettant d'avoir une accuracy supérieur à 99,9%.



## Questions:
* 6. Enfin, mettre en place une méthode (de votre choix) de classification multi-classe;
 * Donner la matrice de confusion et indiquer les languages les plus difficile à distinguer.

In [128]:

dfDE = pd.DataFrame(FeaDE)
dfDE["lang"]=0

dfFR = pd.DataFrame(FeaFR)
dfFR["lang"]=1

dfEN = pd.DataFrame(FeaEN)
dfEN["lang"]=2

dfES = pd.DataFrame(FeaES)
dfES["lang"]=3

df = pd.concat([dfDE, dfFR, dfEN, dfES])
y = df.pop("lang")
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train.head()


945     3
4245    3
3690    1
3716    3
3578    0
Name: lang, dtype: int64

In [134]:
models = {}
for value in y_train.unique():
    print(f"{value} vs all ")
    y_train_1va = y_train.copy()
    y_train_1va[y_train_1va==value]=-1
    y_train_1va[y_train_1va>=0]=1
    y_train_1va[y_train_1va==-1]=0

    y_test_1va = y_test.copy()
    y_test_1va[y_test_1va==value]=-1
    y_test_1va[y_test_1va>=0]=1
    y_test_1va[y_test_1va==-1]=0
    logreg = LogisticRegression(solver="liblinear", penalty=logreg_cv.best_params_["penalty"],                 C=logreg_cv.best_params_['C'])

    logreg.fit(X_train, y_train_1va)
    y_pred = logreg.predict(X_test)
    print(f"Accuracy on test : {accuracy_score(y_test_1va, y_pred)}")
    models[value]=logreg

3 vs all 
Accuracy on test : 1.0
1 vs all 
Accuracy on test : 1.0
0 vs all 
Accuracy on test : 0.9997054491899853
2 vs all 
Accuracy on test : 1.0


In [188]:
# We initialize an array of zeros, which will contain the maximum probabilities seen for every test input winthin each model
previousMaxProbs = np.zeros(y_test.shape[0])
# We initialize an array of zeros which will contain the predicted class
predictions = np.zeros(y_test.shape[0])

# We iterate over the previously built models
for value, model in models.items():
    # We predict the probability for each input to belong to the class (vs the rest)
    currentProbs = model.predict_proba(X_test)
    # We update the max probabilities we've seen in past
    previousMaxProbs = np.maximum(previousMaxProbs, currentProbs[:, 0])
    # If for each input, the probabiliy to belong to this class was over the previous probability, we update the prediction
    predictions[np.equal(currentProbs[:, 0],previousMaxProbs)]=value
print(accuracy_score(y_test, predictions))

L'accuracy de 100% rend cette approche très performante. Etudions la matrice de confusion.

In [190]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[844,   0,   0,   0],
       [  0, 875,   0,   0],
       [  0,   0, 833,   0],
       [  0,   0,   0, 843]], dtype=int64)

Cette approche permet une classification parfaite des données ! C'est une grand réussite.

N'ayant plus de progès à réaliser, je me contente de laisser ce classifieur utilisant la régression linéaire avec une régularisation L1 et un C optimisé comme classifieur.

Je suis cependant curieux de savoir si l'approche "one vs one" permet d'aboutir aux mêmes résultats.

On commence par définir les couples de données qui vont être classifiées ensembles

In [208]:
array = []
for value in y_train.unique():
    for value2 in y_train.unique():
        if(value != value2 and not array.__contains__(sorted([value, value2]))):

            array.append(sorted([value, value2]))
print(array)

[[1, 3], [0, 3], [2, 3], [0, 1], [1, 2], [0, 2]]


Puis on va sur Wikipedia parce qu'on ne sait pas comment ça marche...

In [199]:
[1, 2].sort()

In [202]:
[1, 2].sort()
